Initialization: The RNN is initialized with random weights for each layer (input-to-hidden, hidden-to-hidden, hidden-to-output). Bias terms for the hidden and output layers are initialized to zero.

Forward Pass:

For each input in the sequence, we calculate the hidden state using the input-to-hidden weights, the previous hidden state, and the bias term.
The output is calculated based on the current hidden state and the output weights.
The hidden states and outputs are stored for use in backpropagation.
Backward Pass (BPTT):

We compute the gradient of the loss with respect to each weight and bias by iterating backward through the sequence (backpropagation through time).
We update the weights using the calculated gradients and a learning rate.
Training: The train function iterates through the forward and backward passes for a given number of epochs.

In [ ]:
import numpy as np

class RNN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        # Initialize dimensions
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate

        # Initialize weights
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01  # Input to hidden
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01  # Hidden to hidden
        self.Why = np.random.randn(output_size, hidden_size) * 0.01  # Hidden to output
        self.bh = np.zeros((hidden_size, 1))  # Hidden bias
        self.by = np.zeros((output_size, 1))  # Output bias

    def forward(self, inputs):
        h_prev = np.zeros((self.hidden_size, 1))
        self.hidden_states = [h_prev]

        # Forward pass
        self.outputs = []
        for x in inputs:
            x = x.reshape(-1, 1)
            h_prev = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h_prev) + self.bh)
            y = np.dot(self.Why, h_prev) + self.by
            self.outputs.append(y)
            self.hidden_states.append(h_prev)

        return self.outputs

    def backward(self, inputs, targets):
        dWxh, dWhh, dWhy = np.zeros_like(self.Wxh), np.zeros_like(self.Whh), np.zeros_like(self.Why)
        dbh, dby = np.zeros_like(self.bh), np.zeros_like(self.by)

        # Initialize gradients for h_prev
        dh_next = np.zeros_like(self.hidden_states[0])

        # Backpropagation through time
        for t in reversed(range(len(inputs))):
            dy = self.outputs[t] - targets[t].reshape(-1, 1)
            dWhy += np.dot(dy, self.hidden_states[t + 1].T)
            dby += dy

            dh = np.dot(self.Why.T, dy) + dh_next
            dh_raw = (1 - self.hidden_states[t + 1] ** 2) * dh  # Derivative of tanh
            dbh += dh_raw
            dWxh += np.dot(dh_raw, inputs[t].reshape(1, -1))
            dWhh += np.dot(dh_raw, self.hidden_states[t].T)
            dh_next = np.dot(self.Whh.T, dh_raw)

        # Update weights
        for param, dparam in zip([self.Wxh, self.Whh, self.Why, self.bh, self.by],
                                 [dWxh, dWhh, dWhy, dbh, dby]):
            param -= self.learning_rate * dparam

    def train(self, inputs, targets, epochs=100):
        for epoch in range(epochs):
            outputs = self.forward(inputs)
            self.backward(inputs, targets)
            if epoch % 10 == 0:
                loss = sum((target.reshape(-1, 1) - output) ** 2 for target, output in zip(targets, outputs))
                print(f'Epoch {epoch}, Loss: {np.sum(loss)}')

# Sample input (for sequence length = 3, input size = 2)
inputs = [np.array([0.5, -0.2]), np.array([0.1, 0.8]), np.array([-0.3, 0.2])]
# Sample target output (for sequence length = 3, output size = 1)
targets = [np.array([0.1]), np.array([0.4]), np.array([-0.1])]

# Create and train RNN
rnn = RNN(input_size=2, hidden_size=4, output_size=1, learning_rate=0.01)
rnn.train(inputs, targets, epochs=100)